### Named Tuples - Modifying and Extending

In [1]:
from collections import namedtuple

In [2]:
Point2D = namedtuple('Point2D', 'x y')

The objects generated by `namedtuple` generated classes are **immutable**.

In other words the following will not work:

In [3]:
origin = Point2D(10,0)

In [4]:
origin.x = 0

AttributeError: ignored

However, we may want to "change" the value of one of the coordinates of our `origin` variable.

This is just like strings, we have to create a new version of the tuple, and assign it to the same label.

Suppose we want to change the x-coordinate of our `origin` to something else, but retain whatever the y-coordinate was.

We could do it as follows:

In [5]:
id(origin)

140040044513320

In [6]:
origin = Point2D(0, origin.y)

In [7]:
origin

Point2D(x=0, y=0)

In [8]:
id(origin) # ofcourse the address changes

140040044000960

Of course this could become quite unwieldy when we have a larger number of properties and we only need to change a single item:

In [ ]:
Stock = namedtuple('Stock', 'symbol year month day open high low close')

In [ ]:
djia = Stock('DJIA', 2018, 1, 25, 26_313, 26_458, 26_260, 26_393)

To update the `close` property for example, we could write:

In [ ]:
djia = Stock(djia.symbol, djia.year, djia.month, djia.day, 
                  djia.open, djia.high, djia.low, 26_394)

Now that was quite painful!

We can be a bit more clever about this and use tuple unpacking and argument unpacking as follows:

In [ ]:
*values, _ = djia

We didn't care about the `close` price since we are replacing it, hence the underscore variable name.

And we now have everything else in a list:

In [ ]:
values

['DJIA', 2018, 1, 25, 26313, 26458, 26260]

And now we are going to use the `*` again, but this time to unpack the list into separate arguments when we call the `Stock` initializer:

In [ ]:
djia = Stock(*values, 26_393) 
# or values.append(1) , Stock(*values)

In [ ]:
djia

Stock(symbol='DJIA', year=2018, month=1, day=25, open=26313, high=26458, low=26260, close=26393)

This is much better than our first attempt!

But this approach does not always work, what happens if we want to change a values somewhere in the middle? Or two values?

We cannot do: 
`*first, month, *last = djia`

That would make no sense whatsoever! (and Python will tell you so!)

Maybe slicing and unpacking can work here...

In [ ]:
djia

Stock(symbol='DJIA', year=2018, month=1, day=25, open=26313, high=26458, low=26260, close=26393)

We could try **slicing**:

In [ ]:
djia[:3]

('DJIA', 2018, 1)

In [ ]:
djia[:3] + (26,) + djia[4:]

('DJIA', 2018, 1, 26, 26313, 26458, 26260, 26393)

So now we could use this to create a new StockPrice instance:

In [ ]:
djia2 = Stock(*(djia[:3] + (26,) + djia[4:]))

In [ ]:
djia2

Stock(symbol='DJIA', year=2018, month=1, day=26, open=26313, high=26458, low=26260, close=26393)

This works, but that's quite cumbersome...

And it gets worse - suppose we want to modify the year and day using this approach:

In [ ]:
djia

Stock(symbol='DJIA', year=2018, month=1, day=25, open=26313, high=26458, low=26260, close=26393)

In [ ]:
values = djia[0:1] + (2019,) + djia[2:3] + (26,) + djia[4:]

In [ ]:
values

('DJIA', 2019, 1, 26, 26313, 26458, 26260, 26393)

In [ ]:
djia3 = Stock(*values)

In [ ]:
djia3

Stock(symbol='DJIA', year=2019, month=1, day=26, open=26313, high=26458, low=26260, close=26393)

Or, if you want to avoid unpacking the `values` into the multiple positional arguments required by the `Stock` constructor, we can make us of the `_make` class method that can use an iterable:

In [ ]:
djia4 = Stock._make(values)

In [ ]:
djia4

Stock(symbol='DJIA', year=2019, month=1, day=26, open=26313, high=26458, low=26260, close=26393)

This is really getting too complex.

Fortunately there's a better way!

The namedtuple implementation also provides another instance method called `_replace` which takes keyword-only arguments. That method will make a copy of the current tuple and substitute property values based on the keyword-only arguments passed in.

In [ ]:
djia

Stock(symbol='DJIA', year=2018, month=1, day=25, open=26313, high=26458, low=26260, close=26393)

In [ ]:
id(djia)

2785020879400

In [ ]:
djia5 = djia._replace(year=2019, day=26)

In [ ]:
djia5

Stock(symbol='DJIA', year=2019, month=1, day=26, open=26313, high=26458, low=26260, close=26393)

In [ ]:
djia

Stock(symbol='DJIA', year=2018, month=1, day=25, open=26313, high=26458, low=26260, close=26393)

In [ ]:
id(djia5)

2785020880480

Much better!!

#### Extending Named Tuples

Sometimes we may want to add one or more properties to an existing class without modifying the code for the custom class itself.

Using inheritance is one way to go about it so you may be tempted to do this with named tuples as well, but it's not easy, and there's a cleaner way to do this if all you're after is additional data fields.

Let's say we have a Point class that is for 2D problems:

In [ ]:
Point2D = namedtuple('Point2D', 'x y')

We could easily create a 3D point class as follows:

In [ ]:
Point3D = namedtuple('Point3D', 'x y z')

But if our named tuple has many fields, such as our `Stock` named tuple that's a little more difficult:

In [ ]:
djia

Stock(symbol='DJIA', year=2018, month=1, day=25, open=26313, high=26458, low=26260, close=26393)

Suppose we want to create a new class, say `StockExt`, it would take some effort:

In [ ]:
StockExt = namedtuple('StockExt', 
                      '''symbol year month day open high low 
                      close previous_close''')

Instead we can leverage that `_fields` property:

In [ ]:
Stock._fields

('symbol', 'year', 'month', 'day', 'open', 'high', 'low', 'close')

Remember that the `namedtuple` initializer can handle a list or tuple containing the field names. For example, the one we just retrieved from `_fields`.

Now all we need to do is create a new tuple that contains those fields along with whatever extras we want:

In [ ]:
new_fields = Stock._fields + ('previous_close',)

In [ ]:
new_fields

('symbol',
 'year',
 'month',
 'day',
 'open',
 'high',
 'low',
 'close',
 'previous_close')

And now we can create our new named tuple this way:

In [ ]:
StockExt = namedtuple('StockExt', Stock._fields + ('previous_close',))

In [ ]:
StockExt._fields

('symbol',
 'year',
 'month',
 'day',
 'open',
 'high',
 'low',
 'close',
 'previous_close')

If you did not want to use tuple concatenation for some reason, you could also do it using strings:

In [ ]:
' '.join(Stock._fields) + ' previous_close'

'symbol year month day open high low close previous_close'

In [ ]:
StockExt = namedtuple('StockExt', 
                      ' '.join(Stock._fields) + ' previous_close')

In [ ]:
StockExt._fields

('symbol',
 'year',
 'month',
 'day',
 'open',
 'high',
 'low',
 'close',
 'previous_close')

Now, with this newly extended class, we may want to take one of the "old" named tuple instance (`djia`) and create the extended version of it using the `StockExt` class.

This is also quite simple to do, since named tuples are tuples, and can therefore be unpacked in the arguments of a function call.

In [ ]:
djia

Stock(symbol='DJIA', year=2018, month=1, day=25, open=26313, high=26458, low=26260, close=26393)

In [ ]:
djia_ext = StockExt(*djia, 25_000)

In [ ]:
djia_ext

StockExt(symbol='DJIA', year=2018, month=1, day=25, open=26313, high=26458, low=26260, close=26393, previous_close=25000)

or, we can use the `_make` method:

In [ ]:
djia_ext = StockExt._make(djia + (25_000, ))

In [ ]:
djia_ext

StockExt(symbol='DJIA', year=2018, month=1, day=25, open=26313, high=26458, low=26260, close=26393, previous_close=25000)